# Bài 1: Xây dựng mô hình LeNet. Huấn luyện và đánh giá mô hình LeNet trên 4 độ đo precision, recall và F1-macro (sử dụng Adam làm optimizer).

In [47]:
# Import thư viện cần dùng
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers, models, utils, optimizers
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D,
    Dense, concatenate, Dropout, BatchNormalization, Activation, Rescaling, Add
)

In [20]:
# Tải dữ liệu qua thư viện Keras cung cấp
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [21]:
# Hiển thị kích thước
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (60000, 28, 28)
y_train shape: (60000,)
X_test shape: (10000, 28, 28)


In [22]:
# Chuẩn hóa  dữ liệu: Chuyển giá trị pixel từ [0, 255] về [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [23]:
# Reshape dữ liệu để phù hợp với đầu vào của Conv2D (batch, height, width, channels)
# Thêm một chiều cho kênh màu (grayscale = 1)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [24]:
# Chuyển đổi nhãn sang dạng one-hot encoding
y_train_one_hot = utils.to_categorical(y_train, 10)
y_test_one_hot = utils.to_categorical(y_test, 10)

In [25]:
# Xem lại kích thước các tập
print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {y_train_one_hot.shape}")

Kích thước tập huấn luyện: (60000, 28, 28, 1)
Kích thước tập kiểm tra: (60000, 10)


In [26]:
# Xây dựng mô hình LeNet - 5
def build_lenet_model():
    model = Sequential(name="LeNet-5")
    # Input: 28x28x1

    # Layer 1: Conv2D 6 bộ lọc 5x5, padding 'same' để giữ kích thước 28x28
    model.add(Conv2D(6, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(28, 28, 1)))

    # Layer 2: Average Pooling 2x2 AvgPool, stride 2
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Layer 3: Conv2D 16 bộ lọc 5x5, không padding
    model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))

    # Layer 4: Average Pooling 2x2 AvgPool, stride 2
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
    # Làm phẳng (Flatten)
    model.add(Flatten())

    # Layer 5: FC (120)
    model.add(Dense(120, activation='relu'))

    # Layer 6: FC (84)
    model.add(Dense(84, activation='relu'))

    # Output: FC (10)
    model.add(Dense(10, activation='softmax'))

    return model

model = build_lenet_model()
model.summary()

c:\Users\tienp\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "LeNet-5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)              │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 120)            │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# Complile
optimizer = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

# Huấn luyện mô hinh
history = model.fit(X_train, y_train_one_hot,
                    batch_size=128,
                    epochs=5,
                    validation_data=(X_test, y_test_one_hot),
                    verbose=1)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9472 - loss: 0.1658 - precision: 0.9661 - recall: 0.9359 - val_accuracy: 0.9770 - val_loss: 0.0767 - val_precision: 0.9808 - val_recall: 0.9745
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9822 - loss: 0.0601 - precision: 0.9844 - recall: 0.9802 - val_accuracy: 0.9821 - val_loss: 0.0590 - val_precision: 0.9844 - val_recall: 0.9802
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9856 - loss: 0.0484 - precision: 0.9873 - recall: 0.9842 - val_accuracy: 0.9847 - val_loss: 0.0481 - val_precision: 0.9865 - val_recall: 0.9838
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9882 - loss: 0.0416 - precision: 0.9894 - recall: 0.9870 - val_accuracy: 0.9876 - val_loss: 0.0491 - val_precision: 0.9888 - val_recall: 0.9860
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9890 - loss: 0.0385 - precision: 0.9901 - recall: 0.9879 - val_accuracy: 0.9858 - val_loss: 0.0451 

In [28]:
# Đánh giá kết quả
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test_one_hot, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")

# Lấy dự đoán của mô hình
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# y_test ban đầu
y_true = y_test

# precision, recall, f1-score cho từng lớp và 'macro avg' chính là F1-macro
target_names = [f"Lớp {i}" for i in range(10)]
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))

Test Loss: 0.0451
Test Accuracy: 0.9858
Test Precision: 0.9881
Test Recall: 0.9837
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

       Lớp 0     0.9798    0.9918    0.9858       980
       Lớp 1     0.9938    0.9921    0.9929      1135
       Lớp 2     0.9932    0.9903    0.9918      1032
       Lớp 3     0.9939    0.9752    0.9845      1010
       Lớp 4     0.9918    0.9827    0.9872       982
       Lớp 5     0.9800    0.9865    0.9832       892
       Lớp 6     0.9794    0.9927    0.9860       958
       Lớp 7     0.9770    0.9922    0.9846      1028
       Lớp 8     0.9748    0.9918    0.9832       974
       Lớp 9     0.9928    0.9623    0.9774      1009

    accuracy                         0.9858     10000
   macro avg     0.9857    0.9858    0.9857     10000
weighted avg     0.9859    0.9858    0.9858     10000



* Mô hình LeNet hoạt động rất hiệu quả trên bộ dữ liệu MNIST, cả 4 chỉ số Test Accuracy, Loss,  Precision, Recall đều ở mức tốt. Hiệu suất trên 10 lớp đều có f1 trên 0.977. Mô hình hội tụ rất tốt và đạt hiệu suất gần như tối đa trên bộ dữ liệu MNIST

# Bài 2: Xây dựng mô hình GoogLeNet. Huấn luyện và đánh giá mô hình GoogLeNet trên 4 độ đo precision, recall và F1 (sử dụng Adam làm optimizer)

In [29]:
# Tải dữ liệu qua thư viện Keras cung cấp
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

In [30]:
# Hiển thị kích thước
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3)


In [31]:
# Chuẩn hóa  dữ liệu
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [32]:
# One-hot encoding
y_train_one_hot = utils.to_categorical(y_train, 10)
y_test_one_hot = utils.to_categorical(y_test, 10)

In [33]:
# Xem lại kích thước các tập
print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm tra: {y_train_one_hot.shape}")

Kích thước tập huấn luyện: (50000, 32, 32, 3)
Kích thước tập kiểm tra: (50000, 10)


In [34]:
# Xây dựng Interception Block
def inception_block(x, filters):
    """
    x: đầu vào (tensor)
    filters: tuple gồm (f1, f3r, f3, f5r, f5, fp)
        - f1: số filters của nhánh 1x1
        - f3r, f3: filters của nhánh 1x1 -> 3x3
        - f5r, f5: filters của nhánh 1x1 -> 5x5
        - fp: filters của nhánh pooling -> 1x1
    """
    (f1, f3r, f3, f5r, f5, fp) = filters

    # Nhánh 1: Conv 1x1
    branch1 = Conv2D(f1, (1,1), padding='same', activation='relu')(x)

    # Nhánh 2: 1x1 -> 3x3
    branch2 = Conv2D(f3r, (1,1), padding='same', activation='relu')(x)
    branch2 = Conv2D(f3, (3,3), padding='same', activation='relu')(branch2)

    # Nhánh 3: 1x1 -> 5x5
    branch3 = Conv2D(f5r, (1,1), padding='same', activation='relu')(x)
    branch3 = Conv2D(f5, (5,5), padding='same', activation='relu')(branch3)

    # Nhánh 4: MaxPooling -> 1x1
    branch4 = MaxPooling2D((3,3), strides=(1,1), padding='same')(x)
    branch4 = Conv2D(fp, (1,1), padding='same', activation='relu')(branch4)

    # Gộp 4 nhánh lại
    output = concatenate([branch1, branch2, branch3, branch4], axis=-1)

    return output

# Xây dựng mô hình Mini_GoogLeNet

def build_mini_googlenet():
    input_shape=(32, 32, 3)
    # Input layer
    inputs = Input(shape=input_shape)

    # Layer 1: Conv2D 3x3
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)

    # Layer 2: MaxPooling 2x2
    x = MaxPooling2D((2, 2), strides=2, padding='same')(x)

    # Layer 3: Inception Block 1
    x = inception_block(x, (64, 96, 128, 16, 32, 32))

    # Layer 4: Inception Block 2
    x = inception_block(x, (128, 128, 192, 32, 96, 64))

    # Layer 5: Max Pooling
    x = MaxPooling2D((2, 2), strides=2, padding='same')(x)

    # Layer 6: Inception Block 3
    x = inception_block(x, (192, 96, 208, 16, 48, 64))

    # Layer 7: Global Average Pooling
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # Layer 8: Dropout ngẫu nhiên 40 % neuron để tránh overfitting
    x = Dropout(0.4)(x)

    # Layer 9: FC (256)
    x = Dense(256, activation='relu')(x)

    # Output layer: FC (10)
    outputs = Dense(10, activation='softmax')(x)

    # Tạo mô hình
    model = Model(inputs=inputs, outputs=outputs, name="Mini-GoogLeNet")
    return model

model = build_mini_googlenet()
model.summary()

Model: "Mini-GoogLeNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 32, 32,    │      1,792 │ input_layer_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 16, 16,    │          0 │ conv2d_23[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 16, 16,    │      6,240 │ max_pooling2d_5[… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_27 (Conv2D)  │ (None, 16, 16,    │      1,040 │ max_pooling2d_5[… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 16, 16,    │          0 │ max_pooling2d_5[… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 16, 16,    │      4,160 │ max_pooling2d_5[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_26 (Conv2D)  │ (None, 16, 16,    │    110,720 │ conv2d_25[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_28 (Conv2D)  │ (None, 16, 16,    │     12,832 │ conv2d_27[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_29 (Conv2D)  │ (None, 16, 16,    │      2,080 │ max_pooling2d_6[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 16, 16,    │          0 │ conv2d_24[0][0],  │
│ (Concatenate)       │ 256)              │            │ conv2d_26[0][0],  │
│                     │                   │            │ conv2d_28[0][0],  │
│                     │                   │            │ conv2d_29[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_31 (Conv2D)  │ (None, 16, 16,    │     32,896 │ concatenate_3[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 16, 16,    │      8,224 │ concatenate_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 16, 16,    │          0 │ concatenate_3[0]… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_30 (Conv2D)  │ (None, 16, 16,    │     32,896 │ concatenate_3[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_32 (Conv2D)  │ (None, 16, 16,    │    221,376 │ conv2d_31[0][0]   │
│                     │ 192)              │            │                 

 Total params: 1,037,674 (3.96 MB)

 Trainable params: 1,037,674 (3.96 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

# Huấn luyện mô hình
history = model.fit(X_train, y_train_one_hot,
                    batch_size=128,
                    epochs=10,
                    validation_data=(X_test, y_test_one_hot),
                    verbose=1)

Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 127s 315ms/step - accuracy: 0.3164 - loss: 1.7964 - precision: 0.5980 - recall: 0.0632 - val_accuracy: 0.4564 - val_loss: 1.5017 - val_precision: 0.7703 - val_recall: 0.1425
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 116s 297ms/step - accuracy: 0.4854 - loss: 1.3850 - precision: 0.6887 - recall: 0.2700 - val_accuracy: 0.5364 - val_loss: 1.2487 - val_precision: 0.7112 - val_recall: 0.3558
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 112s 287ms/step - accuracy: 0.5675 - loss: 1.1845 - precision: 0.7338 - recall: 0.3916 - val_accuracy: 0.6008 - val_loss: 1.0945 - val_precision: 0.7642 - val_recall: 0.4562
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 109s 278ms/step - accuracy: 0.6192 - loss: 1.0495 - precision: 0.7594 - recall: 0.4756 - val_accuracy: 0.6554 - val_loss: 0.9505 - val_precision: 0.7924 - val_recall: 0.5165
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 114s 291ms/step - accuracy: 0.6631 - loss: 0.9355 - precision: 0.7834 - recall: 0.5442 - val_accuracy: 0

In [37]:
# Đánh giá kết quả
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test_one_hot, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")

y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = y_test.flatten()

target_names = [f"Lớp {i}" for i in range(10)]
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))

Test Loss: 0.6667
Test Accuracy: 0.7675
Test Precision: 0.8447
Test Recall: 0.7005
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step
              precision    recall  f1-score   support

       Lớp 0     0.7729    0.8030    0.7876      1000
       Lớp 1     0.8399    0.9180    0.8772      1000
       Lớp 2     0.7067    0.7010    0.7038      1000
       Lớp 3     0.6548    0.4950    0.5638      1000
       Lớp 4     0.6667    0.7980    0.7264      1000
       Lớp 5     0.7060    0.7060    0.7060      1000
       Lớp 6     0.8959    0.6540    0.7561      1000
       Lớp 7     0.7513    0.8730    0.8076      1000
       Lớp 8     0.8730    0.8590    0.8659      1000
       Lớp 9     0.8290    0.8680    0.8481      1000

    accuracy                         0.7675     10000
   macro avg     0.7696    0.7675    0.7643     10000
weighted avg     0.7696    0.7675    0.7643     10000



* Mô hình Mini-GoogLNet đạt được kết quả khá tốt trên bộ dữ liệu CIFAR-10 và còn có thể cải thiện nữa nếu số lượng Epoch nhiều hơn cho thấy Inception Block hoạt động hiệu quả, kết quả tổng thể khá tốt với Test Accuracy: 0.7675 và Test Loss = 0.6667. Và F1-score cũng rất ổn (76.43%), thêm nữa macro avg precision (0.7696) và macro avg recall (0.7675) rất gần nhau cho thấy  mô hình cân bằng. Với bộ dữ liệu phức tạp hơn là CIFAR-10, mô hình Mini-GooLNet đã thể hiện được sự hiệu quả của mình so với các mô hình CNN đơn giản.

# Bài 3*: Xây dựng mô hình ResNet-18, đánh giá mô hình ResNet-18 trên bộ dữ liệu VinaFood21 sử dụng các độ đo precision, recall, và F1 (Sử dụng Adam làm optimizer).

In [45]:
# Load dữ liệu
TRAIN_DIR = "C:/Users/tienp/Downloads/VinaFood21/VinaFood21/train" 
TEST_DIR = "C:/Users/tienp/Downloads/VinaFood21/VinaFood21/test"

IMG_SIZE = 224 # Kích thước tiêu chuẩn cho ResNet
BATCH_SIZE = 32
NUM_CLASSES = 21

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False 
)

# Lấy tên các lớp
class_names = train_ds.class_names
print(f"Có {len(class_names)} lớp: {class_names}")

# Chuẩn hóa 
normalization_layer = Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Tối ưu hóa pipeline dữ liệu
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 10044 files belonging to 21 classes.
Found 6682 files belonging to 21 classes.
Có 21 lớp: ['banh-can', 'banh-hoi', 'banh-mi-chao', 'banh-tet', 'banh-trang-tron', 'banh-u', 'banh-uot', 'bap-nuong', 'bo-kho', 'bo-la-lot', 'bot-chien', 'ca-ri', 'canh-kho-qua', 'canh-khoai-mo', 'ga-nuong', 'goi-ga', 'ha-cao', 'hoanh-thanh-nuoc', 'pha-lau', 'tau-hu', 'thit-kho-trung']


In [60]:
# Xây dựng Residual block
def residual_block(x, filters, stride=1):
    shortcut = x  # lưu lại input để cộng skip

    # Nhánh chính
    x = Conv2D(filters, (3,3), strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3,3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    # Nếu kích thước thay đổi (do stride>1), dùng 1x1 conv để điều chỉnh shortcut
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1,1), strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # Cộng skip connection
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def build_resnet_18(input_shape=(224, 224, 3), num_classes=21):
    inputs = Input(shape=input_shape)
    
    # Layer 1: Conv2D 7x7
    x = Conv2D(64, (7, 7), strides=2, padding='same')(inputs)

    # Layer 2: Batch Norm
    x = BatchNormalization()(x)

    # Layer 3: MaxPool 3x3
    x = MaxPooling2D((3, 3), strides=2, padding='same')(x)
    
    # Residual block 1
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    
    # Residual block 2
    x = residual_block(x, 128, stride=2) 
    x = residual_block(x, 128)
    
    # Residual block 3
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)
    
    # Residual block 4
    x = residual_block(x, 512, stride=2)
    x = residual_block(x, 512)
    
    # Layer 4: Global Average Pooling
    x = GlobalAveragePooling2D()(x)

    # Layer 5: FC
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="ResNet-18")
    return model

model = build_resnet_18()
model.summary()

Model: "ResNet-18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_64 (Conv2D)  │ (None, 112, 112,  │      9,472 │ input_layer_8[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_64[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_11    │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_65 (Conv2D)  │ (None, 56, 56,    │     36,928 │ max_pooling2d_11… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_65[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_16       │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_66 (Conv2D)  │ (None, 56, 56,    │     36,928 │ activation_16[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_66[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d_11… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_17       │ (None, 56, 56,    │          0 │ add_8[0][0]       │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_67 (Conv2D)  │ (None, 56, 56,    │     36,928 │ activation_17[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_67[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_68 (Conv2D)  │ (None, 56, 56,    │     36,928 │ activation_18[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_68[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_9 (Add)         │ (None, 56, 56,    │          0 │ batch_normalizat

 Total params: 11,201,685 (42.73 MB)

 Trainable params: 11,192,085 (42.69 MB)

 Non-trainable params: 9,600 (37.50 KB)

In [61]:
# Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

# Huấn luyện mô hình
history = model.fit(train_ds,
                    epochs=20, 
                    validation_data=test_ds
                   )

Epoch 1/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 367s 1s/step - accuracy: 0.2287 - loss: 2.6098 - precision: 0.4747 - recall: 0.0486 - val_accuracy: 0.1308 - val_loss: 3.8986 - val_precision: 0.1930 - val_recall: 0.0771
Epoch 2/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 370s 1s/step - accuracy: 0.3182 - loss: 2.2253 - precision: 0.5880 - recall: 0.1037 - val_accuracy: 0.1821 - val_loss: 3.3773 - val_precision: 0.2939 - val_recall: 0.1012
Epoch 3/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 367s 1s/step - accuracy: 0.3698 - loss: 2.0353 - precision: 0.6400 - recall: 0.1538 - val_accuracy: 0.2113 - val_loss: 3.0487 - val_precision: 0.3586 - val_recall: 0.1190
Epoch 4/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 363s 1s/step - accuracy: 0.4132 - loss: 1.8783 - precision: 0.6702 - recall: 0.2074 - val_accuracy: 0.2119 - val_loss: 3.2118 - val_precision: 0.3398 - val_recall: 0.1368
Epoch 5/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 364s 1s/step - accuracy: 0.4575 - loss: 1.7270 - precision: 0.6888 - recall: 0.2599 - val_accuracy: 0.2475 - val_los

In [64]:
# Đánh giá mô hình 
test_loss, test_acc, test_precision, test_recall = model.evaluate(test_ds, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Precision (Overall): {test_precision:.4f}")
print(f"Test Recall (Overall): {test_recall:.4f}")

y_true = []
y_pred_probs = []

# Lặp qua tập test để lấy nhãn
for x_batch, y_batch in test_ds:
    y_true_batch = np.argmax(y_batch.numpy(), axis=1)
    y_true.extend(y_true_batch)
    
    y_pred_batch_probs = model.predict_on_batch(x_batch)
    y_pred_probs.extend(y_pred_batch_probs)

y_pred = np.argmax(y_pred_probs, axis=1)

print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

Test Loss: 6.2223
Test Accuracy: 0.2861
Test Precision (Overall): 0.3012
Test Recall (Overall): 0.2742
                  precision    recall  f1-score   support

        banh-can     0.2500    0.0249    0.0453       241
        banh-hoi     0.3330    0.5025    0.4005       607
    banh-mi-chao     0.5556    0.0346    0.0651       289
        banh-tet     0.4234    0.4863    0.4527       364
 banh-trang-tron     0.1254    0.5210    0.2021       309
          banh-u     0.6429    0.0511    0.0947       176
        banh-uot     0.3118    0.0946    0.1452       560
       bap-nuong     0.2531    0.2867    0.2689       143
          bo-kho     0.2713    0.5562    0.3647       338
       bo-la-lot     0.2705    0.9301    0.4191       458
       bot-chien     0.2688    0.0906    0.1355       276
           ca-ri     0.2812    0.1053    0.1532       171
    canh-kho-qua     0.7759    0.3103    0.4433       290
   canh-khoai-mo     0.5411    0.7980    0.6449       198
        ga-nuong     0.541

* Kết quả thu được của mô hình ResNet-18 trên bộ dữ liệu VinaFood21 bị overfitting nặng. Khoảng cách lớn giữa train accuracy 96% và test accuracy 28.6% chính là dấu hiệu cho thấy mô hình đã học vẹt chứ không hiểu được đặc trưng thực sự của các món ăn. Lí do bị overfitting vì mô hình ResNet-18 là một mô hình rất lớn mà bộ dữ liệu VinaFood 21 (chỉ khoảng 10000 ảnh) là quá nhỏ để dạy 11.7 triệu tham số của mô hình. Điều nay cho thấy việc huấn luyện một mô hình rất sâu như ResNet-18 trên một bộ dữ liệu nhỏ là một phương pháp không hiệu quả

# Bài 4*: Sử dụng pretrained ResNet50 từ HuggingFace để fine-tune trên bộ dữ liệu VinaFood21.

In [65]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
# Load dữ liệu
TRAIN_DIR = "C:/Users/tienp/Downloads/VinaFood21/VinaFood21/train" 
TEST_DIR = "C:/Users/tienp/Downloads/VinaFood21/VinaFood21/test"

IMG_SIZE = 224 
BATCH_SIZE = 32
NUM_CLASSES = 21

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False 
)

# Chuẩn hóa 
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
test_ds = test_ds.map(lambda x, y: (preprocess_input(x), y))

# Tối ưu hóa pipeline dữ liệu
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 10044 files belonging to 21 classes.
Found 6682 files belonging to 21 classes.


In [66]:
# Xây dựng mô hình ResNet-50

# Tải mô hình ResNet50 gốc 
# weights='imagenet': Tải trọng số đã huấn luyện trên ImageNet
# include_top=False: Bỏ lớp FC 1000 lớp ở cuối
base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Đóng băng toàn bộ mô hình gốc
base_model.trainable = False

# Xây dựng Output mới cho VinaFood21
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Lớp đầu tiên là mô hình ResNet50 đã đóng băng
x = base_model(inputs, training=False) 

# Thêm các lớp của riêng 
# Global Average Pooling
x = GlobalAveragePooling2D()(x) 
# FC 256
x = Dense(256, activation='relu')(x)
# Dropout 0.5
x = Dropout(0.5)(x) 
# Output
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

# Kết hợp lại thành mô hình cuối cùng
model = Model(inputs=inputs, outputs=outputs, name="ResNet50_Transfer_Learning")

model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "ResNet50_Transfer_Learning"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 21)             │         5,397 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,117,653 (92.00 MB)

 Trainable params: 529,941 (2.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [67]:
# Compile mô hình
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

# Huấn luyện mô hình 
history = model.fit(train_ds,
                    epochs=10,
                    validation_data=test_ds
                   )

Epoch 1/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 419s 1s/step - accuracy: 0.4466 - loss: 1.8176 - precision: 0.7333 - recall: 0.2638 - val_accuracy: 0.6449 - val_loss: 1.1347 - val_precision: 0.8367 - val_recall: 0.4539
Epoch 2/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 474s 2s/step - accuracy: 0.6266 - loss: 1.1834 - precision: 0.7976 - recall: 0.4872 - val_accuracy: 0.6814 - val_loss: 0.9872 - val_precision: 0.8293 - val_recall: 0.5548
Epoch 3/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 376s 1s/step - accuracy: 0.6872 - loss: 0.9743 - precision: 0.8230 - recall: 0.5762 - val_accuracy: 0.7109 - val_loss: 0.9194 - val_precision: 0.8359 - val_recall: 0.6043
Epoch 4/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 383s 1s/step - accuracy: 0.7214 - loss: 0.8692 - precision: 0.8372 - recall: 0.6199 - val_accuracy: 0.7145 - val_loss: 0.8983 - val_precision: 0.8281 - val_recall: 0.6299
Epoch 5/10
314/314 ━━━━━━━━━━━━━━━━━━━━ 385s 1s/step - accuracy: 0.7397 - loss: 0.7948 - precision: 0.8464 - recall: 0.6539 - val_accuracy: 0.7109 - val_los

In [68]:
# Đánh giá mô hình
test_loss, test_acc, test_precision, test_recall = model.evaluate(test_ds, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Precision (Overall): {test_precision:.4f}")
print(f"Test Recall (Overall): {test_recall:.4f}")

y_true = []
y_pred_probs = []

# Lặp qua tập test để lấy nhãn
for x_batch, y_batch in test_ds:
    y_true_batch = np.argmax(y_batch.numpy(), axis=1)
    y_true.extend(y_true_batch)
    
    y_pred_batch_probs = model.predict_on_batch(x_batch)
    y_pred_probs.extend(y_pred_batch_probs)

y_pred = np.argmax(y_pred_probs, axis=1)

print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

Test Loss: 0.8273
Test Accuracy: 0.7396
Test Precision (Overall): 0.8181
Test Recall (Overall): 0.6928
                  precision    recall  f1-score   support

        banh-can     0.7380    0.8299    0.7812       241
        banh-hoi     0.6939    0.7694    0.7297       607
    banh-mi-chao     0.8357    0.8097    0.8225       289
        banh-tet     0.9417    0.7995    0.8648       364
 banh-trang-tron     0.8218    0.7314    0.7740       309
          banh-u     0.7653    0.9261    0.8380       176
        banh-uot     0.5698    0.6411    0.6034       560
       bap-nuong     0.8921    0.8671    0.8794       143
          bo-kho     0.7925    0.4970    0.6109       338
       bo-la-lot     0.8806    0.8210    0.8497       458
       bot-chien     0.6953    0.7029    0.6991       276
           ca-ri     0.5441    0.2164    0.3096       171
    canh-kho-qua     0.6582    0.8897    0.7566       290
   canh-khoai-mo     0.9699    0.8131    0.8846       198
        ga-nuong     0.821

Khi sử dụng mô hình ResNet-50 (đã được học từ 1.2 triệu ảnh) thông thái hơn so với mô hình ResNet-18 (chưa  được học). Kết quả thu được trên bộ dữ liệu VinaFood21 tốt hơn rõ ràng với Accuracy đạt được 74%. Với lượng kiến thức khổng lồ đã có trước đó ResNet-50 đã học rất nhanh và hiệu quả